In [11]:
# Importing packages

from selenium import webdriver
from bs4 import BeautifulSoup

import urllib
from urllib.request import urlopen
import re
from time import sleep

import numpy as np
import pandas as pd
from random import randint

from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
import math
import os

import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import warnings 
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [12]:
url = pd.read_csv("article_links.csv")
df = pd.DataFrame(url)
df.shape

(4722, 2)

In [13]:
df.head()

,Unnamed: 0,links
0,1,https://www.straitstimes.com/singapore/top-sto...
1,2,https://www.straitstimes.com/singapore/st-now-...
2,3,https://www.straitstimes.com/singapore/what-el...
3,4,https://www.straitstimes.com/singapore/nus-stu...
4,5,https://www.straitstimes.com/singapore/water-m...


In [14]:
# Seperating links into different category
df['Category'] = np.nan
for i,link in enumerate(df.links):
    if "courts-crime" in link:
        df['Category'][i] = 'Courts & Crime'
    elif "manpower" in link:
        df['Category'][i] = 'Manpower'
    elif "housing" in link:
        df['Category'][i] = 'Housing'
    elif "environment" in link:
        df['Category'][i] = 'Environment'
    elif "education" in link:
        df['Category'][i] = 'Education'
    elif "health" in link:
        df['Category'][i] = 'Health'
    elif "transport" in link:
        df['Category'][i] = 'Transport'
    else:
        df['Category'][i] = 'General'

In [5]:
df.to_csv("article_links_1.csv")

In [15]:
df.Category.value_counts()

General           2061
Courts & Crime    1018
Transport          601
Health             404
Education          335
Environment        137
Manpower            99
Housing             67
Name: Category, dtype: int64

In [16]:
df_business = pd.read_csv("business.csv")
df_business['Category'] = 'Business'
df = df.append(df_business)

In [17]:
df.shape

(7271, 3)

In [7]:
df.sample(50)

,Unnamed: 0,links,Category
4680,4681,https://www.straitstimes.com/singapore/manpowe...,Manpower
2910,2911,https://www.straitstimes.com/singapore/courts-...,Courts & Crime
3303,3304,https://www.straitstimes.com/singapore/courts-...,Courts & Crime
2758,2759,https://www.straitstimes.com/singapore/courts-...,Courts & Crime
4144,4145,https://www.straitstimes.com/singapore/transpo...,Transport
334,335,https://www.straitstimes.com/singapore/1160-ne...,General
3127,3128,https://www.straitstimes.com/singapore/st-glob...,General
3125,3126,https://www.straitstimes.com/asia/se-asia/iden...,General
4640,4641,https://www.straitstimes.com/singapore/housing...,Housing
3833,3834,https://www.straitstimes.com/singapore/educati...,Education


In [10]:
# # Check if all links are appended

# print('Courts-crime:',len(courts_crime))
# print('Education:',len(education))
# print('Housing:',len(housing))
# print('Transport:',len(transport))
# print('Health:',len(health))
# print('Manpower:',len(manpower))
# print('Environment:',len(environment))
# print('General:',len(general))

# print("Total count:",len(courts_crime)+
# len(education)+
# len(housing)+
# len(transport)+
# len(health)+
# len(manpower)+
# len(environment)+
# len(general))

In [11]:
# # Seperating links by respective categories

# df = pd.DataFrame({'links':courts_crime}, index=None).to_csv("courts_crime.csv")
# df = pd.DataFrame({'links':education}, index=None).to_csv("education.csv")
# df = pd.DataFrame({'links':housing}, index=None).to_csv("housing.csv")
# df = pd.DataFrame({'links':transport}, index=None).to_csv("transport.csv")
# df = pd.DataFrame({'links':health}, index=None).to_csv("health.csv")
# df = pd.DataFrame({'links':manpower}, index=None).to_csv("manpower.csv")
# df = pd.DataFrame({'links':environment}, index=None).to_csv("environment.csv")
# df = pd.DataFrame({'links':general}, index=None).to_csv("general.csv")

In [12]:
# url_1 = pd.read_csv("courts_crime.csv")
# url_1.drop('Unnamed: 0',axis=1, inplace=True)

# url_2 = pd.read_csv("education.csv")
# url_2.drop('Unnamed: 0',axis=1, inplace=True)

# url_3 = pd.read_csv("housing.csv")
# url_3.drop('Unnamed: 0',axis=1, inplace=True)

# url_4 = pd.read_csv("transport.csv")
# url_4.drop('Unnamed: 0',axis=1, inplace=True)

# url_5 = pd.read_csv("health.csv")
# url_5.drop('Unnamed: 0',axis=1, inplace=True)

# url_6 = pd.read_csv("manpower.csv")
# url_6.drop('Unnamed: 0',axis=1, inplace=True)

# url_7 = pd.read_csv("environment.csv")
# url_7.drop('Unnamed: 0',axis=1, inplace=True)

# url_8 = pd.read_csv("general.csv")
# url_8.drop('Unnamed: 0',axis=1, inplace=True)

# url_9 = pd.read_csv("business.csv")
# url_9.drop('Unnamed: 0',axis=1, inplace=True)

In [18]:
# Testing web scraping on one random article

source = requests.get("https://www.straitstimes.com/singapore/education/nus-committee-proposes-longer-suspensions-and-expulsion-for-sexual-misconduct").text
soup = BeautifulSoup(source, 'lxml')

# To get the news title
print('Headline:', soup.find('h1', class_="headline node-title").text)

# To get the news publish date
print('Publish Date:', soup.find('div', class_='story-postdate').text[9:])

# To get the Author of the article
print('Author:',soup.find('div', class_='author-field author-name').text)

# To get the author's designation
print("Author's Designation:",soup.find('div', class_='author-designation author-field').text)

# To get the article's content
articles_t = str()
for para in soup.find_all('div', class_='odd field-item'):
    for i in para.find_all('p'):
        articles_t += i.text
print("Sample Content:",articles_t[:115])

# To get the topics tagged to the article
tags_t=[]
for i in soup.find_all('ul'):
    for ii in i.find_all('a', href=re.compile('/tags/')):
        tags_t.append(ii.text)
print('Topics:',tags_t)

Headline: NUS committee proposes longer suspensions and expulsion for sexual misconduct offences
Publish Date: May 15, 2019, 11:12 am SGT
Author: Amelia Teng
Author's Designation: Education Correspondent
Sample Content: SINGAPORE - The committee set up to review how the National University of Singapore (NUS) handles sexual misconduct
Topics: ['NUS', 'SEX OFFENCES']


In [19]:
# Defining a function for reproducibility

def webscraper(df):

    '''To extract out necessary elements from a page of The Straits Times'''
    
    # Containers to store data
    Headline = []
    Publish_Date = []
    Category = []
    Author = []
    Author_Title = []
    Content = []
    Topics = []

    # Extract link to job descriptions
    for i in range(len(df)):
        print('Link Number',i)
        source = requests.get(df.links[i]).text
        soup = BeautifulSoup(source, 'lxml')
        
        Category.append(df.Category[i])

        try:
            news_headline = soup.find('h1', class_="headline node-title").text
        except:
            news_headline = np.nan
            print('Headline Failed')
        Headline.append(news_headline)

        try:
            date = soup.find('div', class_='story-postdate').text[9:]
        except:
            date = np.nan
            print('Date Failed')
        Publish_Date.append(date)

        try:
            author_name = soup.find('div', class_='author-field author-name').text
        except:
            author_name = np.nan
            print('Author Failed')
        Author.append(author_name)

        try:
            a_title = soup.find('div', class_='author-designation author-field').text
        except:
            a_title = np.nan
            print('Author Title Failed')
        Author_Title.append(a_title)

        try:
            articles = str()
            for para in soup.find_all('div', class_='odd field-item'):
                for i in para.find_all('p'):
                    articles += i.text
        except:
            articles = np.nan
            print('Content Failed')
        Content.append(articles)

        try:
            tag=[]
            for i in soup.find_all('ul'):
                for ii in i.find_all('a', href=re.compile('/tags/')):
                    tag.append(ii.text)
        except:
            tag = np.nan
            print('Tags Failed')
        Topics.append(tag)
        
    # Exporting data to csv file for storage
    df1 = pd.DataFrame({
                        'Headline':Headline, 'Publish Date': Publish_Date, 'Category': Category,
                        'Author': Author, 'Author Title': Author_Title,'Content': Content, 'Tags': Topics
    }, index=None).to_csv("all_news.csv")
    
    df1 = pd.read_csv("all_news.csv")
    

In [21]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df.reset_index(inplace=True,drop=True)

In [25]:
df.head()

,links,Category
0,https://www.straitstimes.com/singapore/top-sto...,General
1,https://www.straitstimes.com/singapore/st-now-...,General
2,https://www.straitstimes.com/singapore/what-el...,General
3,https://www.straitstimes.com/singapore/nus-stu...,General
4,https://www.straitstimes.com/singapore/water-m...,General


In [24]:
webscraper(df)

Link Number 0
Author Failed
Author Title Failed
Link Number 1
Author Failed
Author Title Failed
Link Number 2
Author Failed
Author Title Failed
Link Number 3
Author Title Failed
Link Number 4
Author Title Failed
Link Number 5
Author Title Failed
Link Number 6
Author Title Failed
Link Number 7
Link Number 8
Author Title Failed
Link Number 9
Link Number 10
Link Number 11
Link Number 12
Link Number 13
Link Number 14
Author Title Failed
Link Number 15
Link Number 16
Author Title Failed
Link Number 17
Author Title Failed
Link Number 18
Author Title Failed
Link Number 19
Author Title Failed
Link Number 20
Author Title Failed
Link Number 21
Link Number 22
Author Title Failed
Link Number 23
Author Title Failed
Link Number 24
Author Title Failed
Link Number 25
Link Number 26
Author Title Failed
Link Number 27
Link Number 28
Link Number 29
Author Title Failed
Link Number 30
Author Failed
Author Title Failed
Link Number 31
Author Failed
Author Title Failed
Link Number 32
Author Title Failed
Link 

Link Number 281
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 282
Author Title Failed
Link Number 283
Author Title Failed
Link Number 284
Link Number 285
Author Title Failed
Link Number 286
Author Title Failed
Link Number 287
Author Title Failed
Link Number 288
Author Title Failed
Link Number 289
Author Title Failed
Link Number 290
Link Number 291
Link Number 292
Link Number 293
Link Number 294
Link Number 295
Author Title Failed
Link Number 296
Link Number 297
Link Number 298
Author Failed
Author Title Failed
Link Number 299
Link Number 300
Author Failed
Author Title Failed
Link Number 301
Link Number 302
Link Number 303
Link Number 304
Author Failed
Author Title Failed
Link Number 305
Author Failed
Author Title Failed
Link Number 306
Author Title Failed
Link Number 307
Author Title Failed
Link Number 308
Link Number 309
Author Title Failed
Link Number 310
Author Title Failed
Link Number 311
Author Title Failed
Link Number 312
Author Title Failed
Link Numbe

Author Failed
Author Title Failed
Link Number 571
Link Number 572
Author Title Failed
Link Number 573
Link Number 574
Link Number 575
Link Number 576
Link Number 577
Author Failed
Author Title Failed
Link Number 578
Author Title Failed
Link Number 579
Author Title Failed
Link Number 580
Author Title Failed
Link Number 581
Link Number 582
Link Number 583
Link Number 584
Author Title Failed
Link Number 585
Link Number 586
Author Title Failed
Link Number 587
Link Number 588
Link Number 589
Link Number 590
Link Number 591
Link Number 592
Link Number 593
Link Number 594
Link Number 595
Author Failed
Author Title Failed
Link Number 596
Author Failed
Author Title Failed
Link Number 597
Link Number 598
Link Number 599
Link Number 600
Link Number 601
Link Number 602
Link Number 603
Link Number 604
Link Number 605
Link Number 606
Link Number 607
Link Number 608
Author Failed
Author Title Failed
Link Number 609
Author Failed
Author Title Failed
Link Number 610
Author Title Failed
Link Number 611


Link Number 830
Link Number 831
Link Number 832
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 833
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 834
Author Failed
Author Title Failed
Link Number 835
Link Number 836
Author Title Failed
Link Number 837
Link Number 838
Link Number 839
Link Number 840
Author Title Failed
Link Number 841
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 842
Link Number 843
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 844
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 845
Link Number 846
Link Number 847
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 848
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 849
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 850
Author Failed
Author Title Failed
Link Number 851
Headline Failed
Date Failed
Author Failed
Au

Author Title Failed
Link Number 1013
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 1014
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 1015
Author Title Failed
Link Number 1016
Author Failed
Author Title Failed
Link Number 1017
Author Failed
Author Title Failed
Link Number 1018
Author Title Failed
Link Number 1019
Link Number 1020
Link Number 1021
Link Number 1022
Link Number 1023
Link Number 1024
Link Number 1025
Author Title Failed
Link Number 1026
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 1027
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 1028
Author Failed
Author Title Failed
Link Number 1029
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 1030
Author Title Failed
Link Number 1031
Link Number 1032
Author Title Failed
Link Number 1033
Author Title Failed
Link Number 1034
Link Number 1035
Headline Failed
Date Failed
Author Failed
Author Title Fa

...

Author Failed
Author Title Failed
Link Number 6600
Author Failed
Author Title Failed
Link Number 6601
Author Failed
Author Title Failed
Link Number 6602
Link Number 6603
Author Failed
Author Title Failed
Link Number 6604
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 6605
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 6606
Author Failed
Author Title Failed
Link Number 6607
Author Failed
Author Title Failed
Link Number 6608
Author Failed
Author Title Failed
Link Number 6609
Author Failed
Author Title Failed
Link Number 6610
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 6611
Author Failed
Author Title Failed
Link Number 6612
Author Failed
Author Title Failed
Link Number 6613
Author Failed
Author Title Failed
Link Number 6614
Author Failed
Author Title Failed
Link Number 6615
Author Failed
Author Title Failed
Link Number 6616
Author Failed
Author Title Failed
Link Number 6617
Author Failed
Author Title Faile

Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 6753
Author Failed
Author Title Failed
Link Number 6754
Author Failed
Author Title Failed
Link Number 6755
Author Failed
Author Title Failed
Link Number 6756
Author Failed
Author Title Failed
Link Number 6757
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 6758
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 6759
Link Number 6760
Author Failed
Author Title Failed
Link Number 6761
Author Failed
Author Title Failed
Link Number 6762
Author Failed
Author Title Failed
Link Number 6763
Author Failed
Author Title Failed
Link Number 6764
Author Failed
Author Title Failed
Link Number 6765
Author Failed
Author Title Failed
Link Number 6766
Author Failed
Author Title Failed
Link Number 6767
Author Failed
Author Title Failed
Link Number 6768
Author Failed
Author Title Failed
Link Number 6769
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 6770
Auth

Author Failed
Author Title Failed
Link Number 6902
Author Failed
Author Title Failed
Link Number 6903
Author Failed
Author Title Failed
Link Number 6904
Author Failed
Author Title Failed
Link Number 6905
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 6906
Author Failed
Author Title Failed
Link Number 6907
Author Failed
Author Title Failed
Link Number 6908
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 6909
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 6910
Author Failed
Author Title Failed
Link Number 6911
Author Failed
Author Title Failed
Link Number 6912
Author Failed
Author Title Failed
Link Number 6913
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 6914
Author Failed
Author Title Failed
Link Number 6915
Author Failed
Author Title Failed
Link Number 6916
Author Failed
Author Title Failed
Link Number 6917
Author Failed
Author Title Failed
Link Number 6918
Author Failed
Author 

Author Failed
Author Title Failed
Link Number 7059
Author Failed
Author Title Failed
Link Number 7060
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 7061
Author Failed
Author Title Failed
Link Number 7062
Author Failed
Author Title Failed
Link Number 7063
Author Failed
Author Title Failed
Link Number 7064
Author Failed
Author Title Failed
Link Number 7065
Author Failed
Author Title Failed
Link Number 7066
Author Failed
Author Title Failed
Link Number 7067
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 7068
Author Failed
Author Title Failed
Link Number 7069
Author Failed
Author Title Failed
Link Number 7070
Author Failed
Author Title Failed
Link Number 7071
Author Failed
Author Title Failed
Link Number 7072
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 7073
Author Failed
Author Title Failed
Link Number 7074
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 7075
Headline Failed
Date 

Author Failed
Author Title Failed
Link Number 7209
Author Failed
Author Title Failed
Link Number 7210
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 7211
Author Failed
Author Title Failed
Link Number 7212
Author Failed
Author Title Failed
Link Number 7213
Author Failed
Author Title Failed
Link Number 7214
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 7215
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 7216
Author Failed
Author Title Failed
Link Number 7217
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Number 7218
Author Failed
Author Title Failed
Link Number 7219
Author Failed
Author Title Failed
Link Number 7220
Author Failed
Author Title Failed
Link Number 7221
Author Failed
Author Title Failed
Link Number 7222
Author Failed
Author Title Failed
Link Number 7223
Author Failed
Author Title Failed
Link Number 7224
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Numbe

In [32]:
df1 = pd.read_csv("all_news.csv")

In [37]:
df1 = df1.drop("Unnamed: 0",axis=1)
df1.head()

,Headline,Publish Date,Category,Author,Author Title,Content,Tags
0,Top stories from The Straits Times on Wednesda...,"May 15, 2019, 7:15 am SGT",General,NaN,NaN,Good morning! Here are our top stories to kick...,['NEWSLETTER']
1,"ST Now, News As It Happens - May 15, 2019",": May 15, 2019, 5:37 am SGT",General,NaN,NaN,"Welcome to ST Now, News As It Happens. Recap o...",['LIVE BLOG']
2,"What else, other than Sejarah Melayu, can you ...","May 15, 2019, 4:00 am SGT",General,NaN,NaN,All the talk about The First Print exhibition ...,"['ASK NLB', 'NATIONAL LIBRARY BOARD']"
3,NUS approves plans by students' union to hold ...,"May 14, 2019, 11:50 pm SGT",General,Shabana Begum,NaN,SINGAPORE - The National University of Singapo...,"['NUS', 'VOYEURS']"
4,Water management social enterprise is the firs...,"May 14, 2019, 11:15 pm SGT",General,Shabana Begum,NaN,"SINGAPORE - A local water management company, ...","['WATER CONSERVATION', 'AWARDS AND PRIZES']"


In [38]:
df1.Headline.isnull().sum()

1343

In [51]:
def null_retry(df, df_url, null_news):
    
    '''To re-run rows which were not captured previously in the intial function'''
    
    null_index = []
    for i,text in enumerate(df.Headline):
        if type(text) != str:
            null_index.append(i)
    
    Headline = []
    Publish_Date = []
    Category = []
    Author = []
    Author_Title = []
    Content = []
    Topics = []


    for i in null_index:
            print('Link Index',i)
            source = requests.get(df_url.links[i]).text
            soup = BeautifulSoup(source, 'lxml')
            
            Category.append(df.Category[i])

            try:
                news_headline = soup.find('h1', class_="headline node-title").text
            except:
                news_headline = np.nan
                print('Headline Failed')
            Headline.append(news_headline)

            try:
                date = soup.find('div', class_='story-postdate').text[9:]
            except:
                date = np.nan
                print('Date Failed')
            Publish_Date.append(date)

            try:
                author_name = soup.find('div', class_='author-field author-name').text
            except:
                author_name = np.nan
                print('Author Failed')
            Author.append(author_name)

            try:
                a_title = soup.find('div', class_='author-designation author-field').text
            except:
                a_title = np.nan
                print('Author Title Failed')
            Author_Title.append(a_title)

            try:
                articles = str()
                for para in soup.find_all('div', class_='odd field-item'):
                    for i in para.find_all('p'):
                        articles += i.text
            except:
                articles = np.nan
                print('Content Failed')
            Content.append(articles)

            try:
                tag=[]
                for i in soup.find_all('ul'):
                    for ii in i.find_all('a', href=re.compile('/tags/')):
                        tag.append(ii.text)
            except:
                tag = np.nan
                print('Tags Failed')
            Topics.append(tag)
            
            

    # Exporting data to csv file for storage
    df_category = pd.DataFrame({
                        'Headline':Headline, 'Publish Date': Publish_Date, 'Category': Category,
                        'Author': Author, 'Author Title': Author_Title,'Content': Content, 'Tags': Topics
    }, index=None).to_csv(null_news + ".csv")
    

In [ ]:
null_retry(df1, df, "null_news1")

In [143]:
df2 = pd.read_csv("null_news1.csv")
df2.Headline.isnull().sum()

618

In [56]:
# Retrying again 

null_retry(df2, df, "null_news2")

Link Index 5
Author Title Failed
Link Index 8
Author Title Failed
Link Index 15
Link Index 18
Author Title Failed
Link Index 19
Author Title Failed
Link Index 23
Author Title Failed
Link Index 25
Link Index 26
Author Title Failed
Link Index 28
Link Index 35
Author Title Failed
Link Index 38
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Index 40
Link Index 44
Link Index 45
Link Index 49
Link Index 50
Author Title Failed
Link Index 53
Link Index 57
Author Title Failed
Link Index 59
Author Failed
Author Title Failed
Link Index 60
Link Index 63
Link Index 69
Author Title Failed
Link Index 73
Author Title Failed
Link Index 74
Author Title Failed
Link Index 76
Link Index 85
Link Index 86
Author Title Failed
Link Index 90
Author Title Failed
Link Index 91
Author Title Failed
Link Index 94
Link Index 96
Link Index 113
Link Index 119
Author Title Failed
Link Index 122
Author Title Failed
Link Index 128
Author Title Failed
Link Index 130
Headline Failed
Date Failed
Author Fa

Author Failed
Author Title Failed
Link Index 764
Author Title Failed
Link Index 766
Author Title Failed
Link Index 767
Author Title Failed
Link Index 770
Link Index 771
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Index 773
Link Index 775
Author Failed
Author Title Failed
Link Index 776
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Index 779
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Index 781
Author Failed
Author Title Failed
Link Index 784
Link Index 786
Author Title Failed
Link Index 789
Link Index 790
Link Index 791
Link Index 793
Author Title Failed
Link Index 796
Author Title Failed
Link Index 798
Link Index 800
Link Index 801
Author Failed
Author Title Failed
Link Index 802
Author Failed
Author Title Failed
Link Index 803
Link Index 805
Author Title Failed
Link Index 807
Author Failed
Author Title Failed
Link Index 809
Author Title Failed
Link Index 810
Author Title Failed
Link Index 811
Link Index 812
Link Index

Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Index 1174
Author Title Failed
Link Index 1175
Link Index 1176
Author Title Failed
Link Index 1177
Link Index 1178
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Index 1179
Author Title Failed
Link Index 1180
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Index 1181
Link Index 1182
Link Index 1183
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Index 1185
Author Title Failed
Link Index 1186
Link Index 1188
Author Failed
Author Title Failed
Link Index 1190
Link Index 1193
Link Index 1196
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Index 1197
Headline Failed
Date Failed
Author Failed
Author Title Failed
Link Index 1198
Link Index 1200
Link Index 1201
Author Failed
Author Title Failed
Link Index 1202
Link Index 1203
Link Index 1206
Author Title Failed
Link Index 1207
Author Title Failed
Link Index 1208
Link Index 1210
Link Index 1212
Headline

In [144]:
df3 = pd.read_csv("null_news2.csv")
df3.Headline.isnull().sum()

58

In [58]:
null_retry(df3, df, "null_news3")

Link Index 10
Link Index 35
Author Title Failed
Link Index 306
Author Title Failed
Link Index 307
Author Title Failed
Link Index 313
Link Index 316
Author Failed
Author Title Failed
Link Index 317
Link Index 338
Link Index 358
Author Title Failed
Link Index 363
Link Index 379
Link Index 383
Link Index 386
Author Failed
Author Title Failed
Link Index 388
Author Title Failed
Link Index 403
Author Title Failed
Link Index 412
Author Failed
Author Title Failed
Link Index 421
Author Title Failed
Link Index 422
Link Index 430
Link Index 434
Author Failed
Author Title Failed
Link Index 437
Author Title Failed
Link Index 438
Author Title Failed
Link Index 443
Author Title Failed
Link Index 463
Link Index 466
Link Index 470
Link Index 471
Link Index 477
Link Index 482
Link Index 483
Author Failed
Author Title Failed
Link Index 489
Link Index 495
Link Index 497
Author Title Failed
Link Index 500
Author Title Failed
Link Index 513
Link Index 515
Author Title Failed
Link Index 517
Link Index 526
Au

In [145]:
df4 = pd.read_csv("null_news3.csv")
df4.Headline.isnull().sum()

0

In [147]:
df = pd.concat([df1, df2, df3, df4])
df = df.dropna(subset=['Headline'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7271 entries, 0 to 57
Data columns (total 8 columns):
Author          4460 non-null object
Author Title    1723 non-null object
Category        7271 non-null object
Content         7266 non-null object
Headline        7271 non-null object
Publish Date    7271 non-null object
Tags            7271 non-null object
Unnamed: 0      1343 non-null float64
dtypes: float64(1), object(7)
memory usage: 511.2+ KB


In [148]:
df.head()

,Author,Author Title,Category,Content,Headline,Publish Date,Tags,Unnamed: 0
0,NaN,NaN,General,Good morning! Here are our top stories to kick...,Top stories from The Straits Times on Wednesda...,"May 15, 2019, 7:15 am SGT",['NEWSLETTER'],NaN
1,NaN,NaN,General,"Welcome to ST Now, News As It Happens. Recap o...","ST Now, News As It Happens - May 15, 2019",": May 15, 2019, 5:37 am SGT",['LIVE BLOG'],NaN
2,NaN,NaN,General,All the talk about The First Print exhibition ...,"What else, other than Sejarah Melayu, can you ...","May 15, 2019, 4:00 am SGT","['ASK NLB', 'NATIONAL LIBRARY BOARD']",NaN
3,Shabana Begum,NaN,General,SINGAPORE - The National University of Singapo...,NUS approves plans by students' union to hold ...,"May 14, 2019, 11:50 pm SGT","['NUS', 'VOYEURS']",NaN
4,Shabana Begum,NaN,General,"SINGAPORE - A local water management company, ...",Water management social enterprise is the firs...,"May 14, 2019, 11:15 pm SGT","['WATER CONSERVATION', 'AWARDS AND PRIZES']",NaN


In [149]:
# Drop unnecessary column
df = df.drop('Unnamed: 0', axis=1)

# Re-arranging columns 
cols = ['Headline',
 'Publish Date',
 'Category',
 'Author',
 'Author Title',
 'Content',
 'Tags']
df = df[cols]

In [150]:
df.reset_index(drop=True, inplace=True)

In [151]:
df.shape

(7271, 7)

In [154]:
df = df.drop_duplicates()
df.shape

(7173, 7)

In [160]:
# Export to csv
df.to_csv('st_data.csv',index=False)

In [175]:
df.head()

,Headline,Publish Date,Category,Author,Author Title,Content,Tags
0,Top stories from The Straits Times on Wednesda...,"May 15, 2019, 7:15 am SGT",General,NaN,NaN,Good morning! Here are our top stories to kick...,['NEWSLETTER']
1,"ST Now, News As It Happens - May 15, 2019",": May 15, 2019, 5:37 am SGT",General,NaN,NaN,"Welcome to ST Now, News As It Happens. Recap o...",['LIVE BLOG']
2,"What else, other than Sejarah Melayu, can you ...","May 15, 2019, 4:00 am SGT",General,NaN,NaN,All the talk about The First Print exhibition ...,"['ASK NLB', 'NATIONAL LIBRARY BOARD']"
3,NUS approves plans by students' union to hold ...,"May 14, 2019, 11:50 pm SGT",General,Shabana Begum,NaN,SINGAPORE - The National University of Singapo...,"['NUS', 'VOYEURS']"
4,Water management social enterprise is the firs...,"May 14, 2019, 11:15 pm SGT",General,Shabana Begum,NaN,"SINGAPORE - A local water management company, ...","['WATER CONSERVATION', 'AWARDS AND PRIZES']"


In [176]:
df.Content[:2]

0    Good morning! Here are our top stories to kick...
1    Welcome to ST Now, News As It Happens. Recap o...
Name: Content, dtype: object